<a href="https://colab.research.google.com/github/yukinaga/ai_novel/blob/main/section_2/01_ai_novel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIによる小説の執筆
川端康成風の文章を生成しましょう。    
GPT-2のモデルを読み込み、各設定を行った上で小説を執筆します。  

## ライブラリのインストール
GPT-2が含まれるライブラリtransformers、形態素解析（≒単語分割）のためのライブラリsentencepieceをインストールします。

In [ ]:
!pip install transformers
!pip install sentencepiece

## GPT-2の設定

今回は、`rinna/japanese-gpt2-medium `を読み込みます。   

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

## 文章を生成する関数
入力文章から続きの文章を生成する関数を設定します。  
まずは、トークナイザーを使って、始まりの文章をモデルへの入力に変換します。  
`encode()`メソッドでは、`return_tensors`を`"pt"`に設定することで、データがPyTorchのTensor型に変換されるようになります。  
また、`add_special_tokens`を`False`に設定することで、文頭やマスクなどの特殊トークンを除くようになります。  
**参考**: https://huggingface.co/docs/transformers/v4.21.2/en/main_classes/tokenizer#transformers.PreTrainedTokenizer  
  
次に、各設定と共にモデルから文章の生成を行います。  
**generateメソッドの設定**: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate  
今回は一度に複数の文章を生成します。  
モデルの出力を文章に変換し、返り値とします。  

In [ ]:
def getarate_sentences(seed_sentence):
    x = tokenizer.encode(seed_sentence, return_tensors="pt", add_special_tokens=False)  # 入力
    y = model.generate(x, #　入力
                       min_length=50,  # 文章の最小長
                       max_length=100,  # 文章の最大長
                       do_sample=True,   # 次の単語を確率で選ぶ
                       top_k=50, # Top-Kサンプリング
                       top_p=0.95,  # Top-pサンプリング
                       temperature=1.2,  # 確率分布の調整
                       num_return_sequences=3,  # 生成する文章の数
                       pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
                       bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
                       eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
                       bad_word_ids=[[tokenizer.unk_token_id]]  # 生成が許可されないトークンID
                       )  
    generated_sentences = tokenizer.batch_decode(y, skip_special_tokens=True)  # 特殊トークンをスキップして文章に変換
    return generated_sentences

## 文章の生成
川端康成の小説の冒頭をシードにして、GPT-2により小説を執筆します。

In [ ]:
seed_sentence = "国境の長いトンネルを抜けると雪国であった。"  # 川端康成の小説の冒頭
generated_sentences = getarate_sentences(seed_sentence)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

シードの文章にアレンジを加えましょう。

In [ ]:
seed_sentence = "国境の長いトンネルを抜けると大都会であった。"  # シード文章
generated_sentences = getarate_sentences(seed_sentence)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

## 文章の保存
生成した文章を、txtファイルに保存します。  

In [ ]:
with open("ai_yasunari.txt", "w") as f:
    f.write(generated_sentences[0])